In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

Python 3.8.10


In [2]:
import tensorflow as tf

# 只使用CPU进行训练
tf.config.set_visible_devices([], 'GPU')

# 打印Tensorflow版本
print(f"Tensorflow Version: {tf.__version__}")

# 检查是否有可用的GPU设备
if tf.test.is_built_with_cuda():
    print("TensorFlow GPU version is installed")
else:
    print("TensorFlow CPU version is installed")

# 检查TensorFlow是否能够访问GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU devices available:", gpus)
else:
    print("No GPU devices found. Running on CPU.")

# !nvidia-smi

2024-06-04 09:11:22.690215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 09:11:24.161959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tensorflow Version: 2.13.1
TensorFlow GPU version is installed
GPU devices available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# 初始化数据库
db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)

# 初始化基础信息
benchmark = '000016'  # 上证50
train_val_test_period = [
    {'train': ['2009-01-01', '2016-12-31'],
     'val': ['2017-01-01', '2018-12-31'],
     'test': ['2019-01-01', '2024-12-31']}]

# 

In [17]:
class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_reg_label(self, stock_dataframe, N=15):
        stock_df = stock_dataframe.copy()
        # 计算未来N日内的最高收益率
        stock_df['max_return'] = stock_df['close'].rolling(window=N).max().shift(-N) / stock_df['close'] - 1
        # 计算未来N日内的最低收益率
        stock_df['min_return'] = stock_df['close'].rolling(window=N).min().shift(-N) / stock_df['close'] - 1
        # 计算未来N日内的收益率和（期望最高收益率越高越好，最低收益率也越高越好，由于最低收益率是负数，因此使用最高+最低来作为综合收益指标）
        stock_df['return_sum'] = stock_df['max_return'] + stock_df['min_return']
        stock_df['label'] = stock_df['return_sum']
        # 过滤第二天一字涨停股票
        stock_df = stock_df[stock_df['high'].shift(-1) != stock_df['low'].shift(-1)]
        return stock_df[['datetime', 'label']]

    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_history = self.db_downloader._download_history_base_info(stock_code, start_date, end_date)
        stock_profile = self.db_downloader._download_all_stock_info(stock_code)
        stock_indicator = self.db_downloader._download_history_indicator_info(stock_code, start_date, end_date)
        stock_indicator = stock_indicator.replace("", np.NaN).ffill()
        stock_factor_date = self.db_downloader._download_history_date_factor_info(start_date, end_date)
        stock_factor_alpha158 = self.db_downloader._download_history_alpha184_factor_info(stock_code, start_date, end_date)
        stock_label = self._build_reg_label(stock_history)

        stock_df = stock_history.merge(stock_profile, on=['code']) \
                .merge(stock_indicator, on=['code', 'datetime']) \
                .merge(stock_factor_alpha158, on=['code', 'datetime']) \
                .merge(stock_factor_date, on=['datetime']) \
                .merge(stock_label, on=['datetime'])
        stock_df = stock_df.dropna()
        return stock_df
    
    def _get_index_latest_cons(self, benchmark):
        import baostock as bs
        bs.login()
        if benchmark == '000016':
            dataframe = bs.query_sz50_stocks().get_data()
        elif benchmark == '000300':
            dataframe = bs.query_hs300_stocks().get_data()
        bs.logout()
        return dataframe['code'].unique()

    def _process_all_stock(self, benchmark, start_date, end_date):
        stock_code_list = self._get_index_latest_cons(benchmark)
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {benchmark} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)

In [19]:
def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date):
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)
    test_start_date = pd.to_datetime(test_start_date)
    test_end_date = pd.to_datetime(test_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]
    test_data = df[(pd.to_datetime(df['datetime']) >= test_start_date) & (pd.to_datetime(df['datetime']) <= test_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    print(f"test_data_size: {test_data.shape}")
    return train_data, val_data, test_data

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=False, batch_size=32):
    features = dataframe[feature_cols]
    labels = dataframe[label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(features), 10000), seed=1024)
    ds = ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, RobustScaler


class QuantileClipTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        
    def fit(self, X, y=None):
        # 计算给定分位数的分界值
        self.lower_bound_ = np.nanquantile(X, self.lower_quantile, axis=0)
        self.upper_bound_ = np.nanquantile(X, self.upper_quantile, axis=0)
        return self
    
    def transform(self, X, y=None):
        # 对整个数组应用剪辑操作
        return np.clip(X, self.lower_bound_, self.upper_bound_)

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [21]:


feature_config = {
    "target_features": ['label'],
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['month'],
    "string_categorical_features": ['industry', 'season'],
}
batch_size = 1024

关闭滚动回测...


[{'train': ['2009-01-01', '2016-12-31'],
  'val': ['2017-01-01', '2018-12-31'],
  'test': ['2019-01-01', '2024-12-31']}]

In [10]:
date_period_params = backtest_period[0]
print(date_period_params)
train_start_date, train_end_date = date_period_params['train']
val_start_date, val_end_date = date_period_params['val']
test_start_date, test_end_date = date_period_params['test']
# 获取全区间数据
print("开始加载原始数据...")
df = proprocessor._process_all_stock(benchmark=benchmark, start_date=train_start_date, end_date=test_end_date)
# 抽取训练验证数据
print("开始拆分训练、验证、测试集合...")
train_data, val_data, test_data = extract_train_val_data(df, *[train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date])
# 从data中抽取相关特征数据
print("开始抽取特征数据...")
feature_columns = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
label_columns = feature_config.get('target_features', [])
full_feature_columns = feature_columns + label_columns
train_df, val_df, test_df = train_data[full_feature_columns], val_data[full_feature_columns], test_data[full_feature_columns]
# 对相关特征进行特征工程
print("开始对特征进行预处理...")
feature_preprocess_pipeline = Pipeline(steps=[
    ('robust_scaler', RobustScaler()),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_feature_columns = feature_config.get('numeric_features', [])
train_df[preprocess_feature_columns] = feature_preprocess_pipeline.fit_transform(train_df[preprocess_feature_columns])
val_df[preprocess_feature_columns] = feature_preprocess_pipeline.transform(val_df[preprocess_feature_columns])
test_df[preprocess_feature_columns] = feature_preprocess_pipeline.transform(test_df[preprocess_feature_columns])

print("开始对标签进行预处理...")
label_preprocess_pipeline = Pipeline(steps=[
    ('quantile_clipper', QuantileClipTransformer()),
    ('robust_scaler', RobustScaler()),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_target_columns = feature_config.get('target_features', [])
train_df[preprocess_target_columns] = label_preprocess_pipeline.fit_transform(train_df[preprocess_target_columns])
val_df[preprocess_target_columns] = label_preprocess_pipeline.transform(val_df[preprocess_target_columns])
test_df[preprocess_target_columns] = label_preprocess_pipeline.transform(test_df[preprocess_target_columns])

{'train': ['2009-01-01', '2016-12-31'], 'val': ['2017-01-01', '2018-12-31'], 'test': ['2019-01-01', '2024-12-31']}
开始加载原始数据...
login success!


Process: 000016 ...: 100%|██████████| 50/50 [00:08<00:00,  6.14it/s]


开始拆分训练、验证、测试集合...
train_data_size: (50924, 204)
validation_data_size: (18461, 204)
test_data_size: (59552, 204)
开始抽取特征数据...
开始对特征进行预处理...
开始对标签进行预处理...


In [12]:
# 转换为tensorflow所使用的dataset
print("开始将DataFrame转换为DataSet...")
train_ds = df_to_dataset(train_df, feature_columns, label_columns, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)  

开始将DataFrame转换为DataSet...


In [13]:
# 准备模型训练
print("开始模型初始化 & 训练...")
from models.single_task.model_moe import QuantModel
model_config = {
        "seed": 1024,
        "feature_use_embedding": False,
        "feature_embedding_dims": 4,
        "numeric_features_with_boundaries": {k: pd.qcut(train_df[k], q=20, retbins=True, duplicates='drop')[1].tolist() for k in feature_config.get('numeric_features', [])},
        "integer_categorical_features_with_vocab": {k: list(train_df[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
        "string_categorical_features_with_vocab": {k: list(train_df[k].unique()) for k in feature_config.get('string_categorical_features', [])},
    }
model = QuantModel(config=model_config)

initial_learning_rate = 5e-4

model.compile(
    optimizer=tf.keras.optimizers.Adam(initial_learning_rate),
    loss=tf.keras.losses.Huber(),
    )

EPOCHS = 500

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

baseline_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks = [early_stopping],
)

开始模型初始化 & 训练...
Epoch 1/500


2024-05-30 05:06:46.507021: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14b75c1d5880 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-30 05:06:46.507173: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-05-30 05:06:46.546793: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-30 05:06:46.696585: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 [==============================] - 15s 191ms/step - loss: 0.4610 - val_loss: 0.0761
Epoch 2/500
50/50 [==============================] - 5s 91ms/step - loss: 0.2794 - val_loss: 0.0466
Epoch 3/500
50/50 [==============================] - 5s 97ms/step - loss: 0.2065 - val_loss: 0.0371
Epoch 4/500
50/50 [==============================] - 5s 94ms/step - loss: 0.1626 - val_loss: 0.0285
Epoch 5/500
50/50 [==============================] - 5s 93ms/step - loss: 0.1266 - val_loss: 0.0236
Epoch 6/500
50/50 [==============================] - 4s 90ms/step - loss: 0.0990 - val_loss: 0.0201
Epoch 7/500
50/50 [==============================] - 5s 94ms/step - loss: 0.0816 - val_loss: 0.0181
Epoch 8/500
11/50 [=====>........................] - ETA: 3s - loss: 0.0697

50/50 [==============================] - 5s 109ms/step - loss: 0.0707 - val_loss: 0.0168
Epoch 9/500
50/50 [==============================] - 4s 88ms/step - loss: 0.0611 - val_loss: 0.0160
Epoch 10/500
50/50 [==============================] - 5s 93ms/step - loss: 0.0535 - val_loss: 0.0154
Epoch 11/500
50/50 [==============================] - 4s 84ms/step - loss: 0.0484 - val_loss: 0.0150
Epoch 12/500
50/50 [==============================] - 5s 90ms/step - loss: 0.0424 - val_loss: 0.0146
Epoch 13/500
50/50 [==============================] - 4s 81ms/step - loss: 0.0384 - val_loss: 0.0143
Epoch 14/500
50/50 [==============================] - 4s 83ms/step - loss: 0.0348 - val_loss: 0.0140
Epoch 15/500
50/50 [==============================] - 4s 85ms/step - loss: 0.0322 - val_loss: 0.0138
Epoch 16/500
50/50 [==============================] - 5s 103ms/step - loss: 0.0298 - val_loss: 0.0135
Epoch 17/500
50/50 [==============================] - 5s 91ms/step - loss: 0.0282 - val_loss: 0.0133
Ep

In [14]:
baseline_history.history.keys()

dict_keys(['loss', 'val_loss'])

In [15]:
# plot_metrics(baseline_history)

In [16]:
# model.save(f'./tf_models/{benchmark}/')

In [17]:
# 输出回测预测
print("开始保存回测预测结果...")
model_pred_result = model.predict(test_ds)
output_df = test_data[['code', 'code_name', 'datetime']]
output_df['label'] = test_df['label']
output_df['label_pred'] = model_pred_result
output_df = output_df.rename(columns={
    'code': 'stock_code',
    'code_name': 'stock_name'
})
output_df.to_pickle(f'../../Offline/backtest/backtest_data/test/{benchmark}_{test_start_date}_回归任务_v6.pkl') 

开始保存回测预测结果...
59/59 [==============================] - 7s 108ms/step


In [18]:
# output_df.head()

In [4]:
import pandas as pd
df = pd.read_pickle('../../Offline/backtest/backtest_data/test/000016_2019-01-01_回归任务_v5.pkl')

In [5]:
def get_model_prediction(model_prediction):
    def get_stock_for_buy(group):
        # group = group[group["label_pred"] > self.params.buy_pred_upper_bound]  # 使用模型预测打分的分布（> 0.9的quantile）进行买入过滤
        select_n = group.nlargest(3, "label_pred")
        return select_n.to_dict("records")

    def get_stock_for_sell(group):
        # group = group[group["label_pred"] < self.params.sell_pred_lower_bound]  # 使用模型预测打分的分布（< 0.1的quantile）进行卖出过滤
        select_n = group.nsmallest(3, "label_pred")
        return select_n.to_dict("records")

    stock_for_buy = model_prediction.groupby("datetime").apply(get_stock_for_buy).to_dict()
    stock_for_sell = model_prediction.groupby("datetime").apply(get_stock_for_sell).to_dict()
    return stock_for_buy, stock_for_sell

In [6]:
stock_for_buy, stock_for_sell = get_model_prediction(df)

In [9]:
# stock_for_buy['2019-01-02']